### Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

### Wysłanie żądania dostępu do strony z opiniami o produkcie

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)
response.status_code

In [ ]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, "html.parser")
    reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")

### Dla każdej z opinii wydobycie z kodu HTML poszczególnych składowych i zapisanie ich w postaci złożonych struktur danych

In [ ]:
for review in reviews:
    try:
        single_review = {
            "review_id": review["data-entry-id"],
            "author": review.select_one("span.user-post__author-name").text.strip(),
            "recomendation": review.select_one("span.user-post__author-recomendation > em").text.strip(),
            "stars": review.select_one("span.user-post__score-count").text.strip(),
            "content": review.select_one("div.user-post__text").text.strip(),
            "pros": [p.text.strip() for p in review.select("div.review-feature__item--positive")],
            "cons": [c.text.strip() for c in review.select("div.review-feature__item--negative")],
            "likes": review.select_one("button.vote-yes > span").text.strip(),
            "dislikes": review.select_one("button.vote-no > span").text.strip(),
            "publish_date": review.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
            "purchase_date": review.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip()
        }
        print(single_review)
    except (AttributeError, TypeError):
        pass
    